In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
#pandas が DataFrame を表示するとき、最大で 20 行まで表示する設定
pd.set_option('display.max_rows', 20)

dt = pd.read_csv('artists.csv')

cols = [
    "This_year's_Top_MV_goods",
    "Top_MV_goods",
    "Spotify_monthly_listeners",
    "This_year's_Top_spotify_streams",
    "Top_spotify_streams",
]

for col in cols:
    dt[col] = dt[col].fillna(dt[col].median())

X = dt.drop(columns=["groupname", "kohaku_selected"]).values
y = dt["kohaku_selected"].values

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
classifier.fit(X_train, y_train)

#正解率の求め方 モデル同士のざっくり比較
from sklearn.metrics import accuracy_score
y_pred = classifier.predict(X_test)
print(accuracy_score(y_test, y_pred))

#問題ごとの「どこを間違えたか」

"""
	   予測：出ない：出る
実際：出ない	TN	FP
実際：出る	  FN	TP

"""

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

"""
p値：
👉 「この説明変数、本当に意味あるの？」を調べるもの

正解率：
👉 「この予測、どれくらい当たったの？」を調べるもの
"""

#y_predは1次元配列だから横に並べるには2次元にする必要がある➡reshape
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))


#今年のMVのいいね数、ストリーミング視聴者、再生数、ticktokの投稿数が関係
#1番は今年のMVのいいね数、次はストリーミング再生数、視聴者数です
#好きなグループのデータ
print(classifier.predict(sc.transform([[656000,110000,170000,405000,5344996,
                                        17847600,7510,7510,4]])))

"""
ロジスティック回帰を用いて、アーティストが紅白歌合戦に選出されるかどうかを
複数の数値指標から予測する分類モデルを作成した。
欠損値を中央値で補完し、標準化を行うことで、各指標を同じスケールで扱えるようにした点が重要であった。

学習データとテストデータに分割することで、モデルの汎化性能を確認し、
正解率だけでなく混同行列を用いて「どのような誤分類が起きているのか」を具体的に把握できた。
その結果、単に当たっているかどうかだけでなく、
「出るはずのアーティストを落としていないか」「出ないはずのアーティストを誤って選んでいないか」
という視点が重要であることを理解した。

また、今年のYouTubeでのMVのいいね数やSpotifyでの再生数・リスナー数など、
直近の人気を表す指標が予測に強く影響していることが読み取れ、
紅白選出が“過去の実績”よりも“現在の勢い”を重視している可能性を考察できた。

一方で、データ数が少なく、データ収集に大きな時間と労力がかかることも実感した。
正解率は約0.71と一定の性能は得られたが、
データが少ない分類問題では結果が不安定になりやすいことも分かった。

本コードを通じて、ロジスティック回帰による分類モデルの基本的な流れ
（前処理・学習・評価・予測）を一通り経験でき、
「モデルを作ること」だけでなく「結果をどう解釈するか」が
データ分析において重要であることを学ぶ良い練習となった。

"""


0.7142857142857143
[[4 1]
 [1 1]]
[[1 1]
 [1 0]
 [0 0]
 [0 0]
 [0 1]
 [0 0]
 [0 0]]
[0]
